# Limpieza y Normalización de los DataFrames
Vamos a limpiar y normalizar tanto los datos de entrenamiento 'train' como los de 'test'.

| | original data | normalized data |
|-------------------|---------------|-----------------|
| training data | dataset_train.csv | normal_train.csv |
| test data | dataset_test.csv | normal_test.csv 

Diferencias entre los datos de entrenamiento y de test originales:
- Los datos de entrenamiento tienen 1600 registros y los de test tienen 400 registros.
- Los datos de test tienen vacía la columna 'Hogwarts House' que es la columa objetivo de la clasificación.

Trabajaremos con una versión reducida (lite5) que contiene:
- Las características base:
    1. 'Best Hand'
    2. 'Age'
- Las 5 asignaturas principales:
    1. 'Defense Against the Dark Arts'
    2. 'Herbology'
    3. 'Potions'
    4. 'Charms'
    5. 'Flying'
- Para los datos de entrenamiento también se incluyen las variables dummy de 'Hogwarts House'. 

In [1]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
from datetime import datetime

# Definición de las características que vamos a mantener
BASE_FEATURES = ['Best Hand', 'Age']
LITE5_COURSES = ['Defense Against the Dark Arts', 'Herbology',
                 'Potions', 'Charms', 'Flying'
]

## Lectura y preparación inicial de los datos

Leemos los datasets originales y calculamos la edad a partir de la columna Birthday.

In [2]:
# Lectura de los datasets
file_train = '../datasets/dataset_train.csv'
file_test = '../datasets/dataset_test.csv'

df_train = pd.read_csv(file_train, index_col=0)
df_test = pd.read_csv(file_test, index_col=0)

## Imputación de datos faltantes usando 'Astronomy'
Usamos la correlación perfecta r=-1 entre 'Astronomy' y 'Defense Against the Dark Arts' para realizar la imputación de los datos faltantes que se puedan imputar de 'Defense Against the Dark Arts' aprovechando esta característica.

Lo único que hay que hacer es tomar el dato de 'Astronomy' y multiplicar por -0.01 para lograr conseguir el dato faltante de  'Defense Against the Dark Arts'.

In [3]:
# Imputar 'Defense Against the Dark Arts' usando 'Astronomy'

def data_imputation_perfect_correlation(df):
    # Calcular las filas totales
    total = len(df)
    
    # Calcular las filas con datos no nulos antes de la imputación
    n1 = df['Defense Against the Dark Arts'].count()
    
    # Crear máscara para identificar:
    # 1. Valores nulos en Defense Against the Dark Arts
    # 2. Valores no nulos en Astronomy
    mask = (df['Defense Against the Dark Arts'].isna() & 
            df['Astronomy'].notna())
    
    # Imputar los valores usando la relación perfecta (Astronomy * -0.01)
    df.loc[mask, 'Defense Against the Dark Arts'] = df.loc[mask, 'Astronomy'] * -0.01
    
    # Calcular las filas con datos no nulos después de la imputación
    n2 = df['Defense Against the Dark Arts'].count()
    
    # Verificar el resultado
    print("Valores no nulos en Defense Against the Dark Arts:")
    print(f"\tAntes de imputación: {n1}/{total}")
    print(f"\tDespués de imputación: {n2}/{total}")
    
    print(f"\nHemos recuperado {n2-n1} filas.")

# Aplicación a los dos DataFrame
print(f"{'='*13} Para el DataFrame train {'='*13}")
data_imputation_perfect_correlation(df_train)
print()
print("="*13, "Para el DataFrame test", "="*13)
data_imputation_perfect_correlation(df_test)

============= Para el DataFrame train =============
Valores no nulos en Defense Against the Dark Arts:
	Antes de imputación: 1569/1600
	Después de imputación: 1600/1600

Hemos recuperado 31 filas.

============= Para el DataFrame test =============
Valores no nulos en Defense Against the Dark Arts:
	Antes de imputación: 392/400
	Después de imputación: 399/400

Hemos recuperado 7 filas.


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1600 entries, 0 to 1599
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Hogwarts House                 1600 non-null   object 
 1   First Name                     1600 non-null   object 
 2   Last Name                      1600 non-null   object 
 3   Birthday                       1600 non-null   object 
 4   Best Hand                      1600 non-null   object 
 5   Arithmancy                     1566 non-null   float64
 6   Astronomy                      1568 non-null   float64
 7   Herbology                      1567 non-null   float64
 8   Defense Against the Dark Arts  1600 non-null   float64
 9   Divination                     1561 non-null   float64
 10  Muggle Studies                 1565 non-null   float64
 11  Ancient Runes                  1565 non-null   float64
 12  History of Magic               1557 non-null   float6

In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, 0 to 399
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Hogwarts House                 0 non-null      float64
 1   First Name                     400 non-null    object 
 2   Last Name                      400 non-null    object 
 3   Birthday                       400 non-null    object 
 4   Best Hand                      400 non-null    object 
 5   Arithmancy                     387 non-null    float64
 6   Astronomy                      387 non-null    float64
 7   Herbology                      389 non-null    float64
 8   Defense Against the Dark Arts  399 non-null    float64
 9   Divination                     394 non-null    float64
 10  Muggle Studies                 390 non-null    float64
 11  Ancient Runes                  392 non-null    float64
 12  History of Magic               389 non-null    float64


## Cálculo de la edad
Es importante que la fecha que se tome como base (reference_date) para el cálculo de la edad en el DataFrame de entrenamiento se guarde en una variable para luego usarla como base en el cálculo de la edad en el DataFrame de test.

In [6]:
# Primero, convertimos Birthday a datetime en ambos datasets
df_train['Birthday'] = pd.to_datetime(df_train['Birthday'])
df_test['Birthday'] = pd.to_datetime(df_test['Birthday'])

# Calculamos la fecha de referencia usando solo el conjunto de entrenamiento
reference_date = df_train['Birthday'].max()

# Definimos la función para calcular la edad
def calculate_age(df, reference_date):
    df['Age'] = (reference_date - df['Birthday']).dt.days / 365.25
    return df

# Aplicamos el cálculo de edad a ambos datasets
df_train = calculate_age(df_train, reference_date)
df_test = calculate_age(df_test, reference_date)

## Eliminación de columnas innecesarias
Incluyendo 'Astronomy' después de usarla para la imputación

In [7]:
# Eliminamos las columnas que no necesitamos
columns_to_drop = ['First Name', 'Last Name', 'Birthday', 'Astronomy']
df_train = df_train.drop(columns=columns_to_drop)
df_test = df_test.drop(columns=columns_to_drop)

## Conversión de datos categóricos a numéricos
Convertimos 'Best Hand' a valores numéricos de tipo `float`.

In [8]:
# Convertimos Best Hand a valores numéricos en ambos datasets
hand_mapping = {'Left': 0, 'Right': 1}
df_train['Best Hand'] = df_train['Best Hand'].map(hand_mapping)
df_test['Best Hand'] = df_test['Best Hand'].map(hand_mapping)

# Convertimos Best Hand a float en ambos datasets
df_train['Best Hand'] = df_train['Best Hand'].astype(float)
df_test['Best Hand'] = df_test['Best Hand'].astype(float)

# Verificamos la conversión
print("Valores únicos en Best Hand (train):", df_train['Best Hand'].unique())
print("Valores únicos en Best Hand (test):", df_test['Best Hand'].unique())

Valores únicos en Best Hand (train): [0. 1.]
Valores únicos en Best Hand (test): [1. 0.]


## Tratamiento específico para los datos de entrenamiento

Para los datos de entrenamiento necesitamos hacer one-hot encoding de la columna 'Hogwarts House'.

In [9]:
# One-hot encoding solo para los datos de entrenamiento
df_train = pd.get_dummies(df_train, columns=['Hogwarts House'], prefix='House', dtype=float)

# Actualizamos BASE_FEATURES para incluir las columnas dummy de House
HOUSE_FEATURES = ['House_Gryffindor', 'House_Hufflepuff', 'House_Ravenclaw', 'House_Slytherin']
BASE_FEATURES = BASE_FEATURES + HOUSE_FEATURES

## Selección de características
Seleccionamos solo las características de lite5 para luego normalizar las columnas numéricas.

In [10]:
# Seleccionamos las columnas que queremos mantener
df_train = df_train[LITE5_COURSES + BASE_FEATURES]
df_test = df_test[LITE5_COURSES + ['Best Hand', 'Age']]  # Note que no incluimos HOUSE_FEATURES para test

In [11]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1600 entries, 0 to 1599
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Defense Against the Dark Arts  1600 non-null   float64
 1   Herbology                      1567 non-null   float64
 2   Potions                        1570 non-null   float64
 3   Charms                         1600 non-null   float64
 4   Flying                         1600 non-null   float64
 5   Best Hand                      1600 non-null   float64
 6   Age                            1600 non-null   float64
 7   House_Gryffindor               1600 non-null   float64
 8   House_Hufflepuff               1600 non-null   float64
 9   House_Ravenclaw                1600 non-null   float64
 10  House_Slytherin                1600 non-null   float64
dtypes: float64(11)
memory usage: 150.0 KB


In [12]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, 0 to 399
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Defense Against the Dark Arts  399 non-null    float64
 1   Herbology                      389 non-null    float64
 2   Potions                        390 non-null    float64
 3   Charms                         400 non-null    float64
 4   Flying                         400 non-null    float64
 5   Best Hand                      400 non-null    float64
 6   Age                            400 non-null    float64
dtypes: float64(7)
memory usage: 25.0 KB


In [13]:
df_train.head(124)

,Defense Against the Dark Arts,Herbology,Potions,Charms,Flying,Best Hand,Age,House_Gryffindor,House_Hufflepuff,House_Ravenclaw,House_Slytherin
Index,,,,,,,,,,,
0,4.878861,5.727180,3.790369,-232.79405,-26.89,0.0,1.598905,0.0,0.0,1.0,0.0
1,5.520605,-5.987446,7.248742,-252.18425,-113.45,1.0,2.058864,0.0,0.0,0.0,1.0
2,3.660761,7.725017,8.728531,-227.34265,30.42,0.0,2.004107,0.0,0.0,1.0,0.0
3,-6.977428,-6.497214,0.821911,-256.84675,200.64,0.0,1.210130,1.0,0.0,0.0,0.0
4,-4.367752,-7.820623,4.311066,-256.38730,157.98,0.0,3.104723,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
119,-4.023001,3.530123,4.274640,-242.27829,-47.35,0.0,4.892539,0.0,1.0,0.0,0.0
120,6.051691,3.649099,8.997370,-230.00220,24.51,0.0,2.091718,0.0,0.0,1.0,0.0
121,4.831771,NaN,12.952911,-249.85396,-75.77,0.0,3.871321,0.0,0.0,0.0,1.0


### Eliminación de las filas con datos faltantes en 'train'
- Se eliminan solo las filas de las características con las que se trabaja y solo del DataFrame 'df_train'.
- En el 'df_test' no se elimina ninguna fila ya que luego aplicaremos un algoritmo de imputación para completar los datos faltantes. 

In [14]:
# Filas de los DataFrame previos a la eliminación de filas faltantes
n_train = len(df_train)

df_train = df_train.dropna()

print(f"El DataFrame 'train' ha pasado de {n_train} a {len(df_train)} filas.")
print(f"Se han eliminado {n_train - len(df_train)} registros con datos faltantes.")
print(f"Se han eliminado solo el {n_train / len(df_train) - 1:.1%} de los registros, por este motivo no realizamos imputación de datos en 'train'.")

El DataFrame 'train' ha pasado de 1600 a 1537 filas.
Se han eliminado 63 registros con datos faltantes.
Se han eliminado solo el 4.1% de los registros, por este motivo no realizamos imputación de datos en 'train'.


In [15]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1537 entries, 0 to 1599
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Defense Against the Dark Arts  1537 non-null   float64
 1   Herbology                      1537 non-null   float64
 2   Potions                        1537 non-null   float64
 3   Charms                         1537 non-null   float64
 4   Flying                         1537 non-null   float64
 5   Best Hand                      1537 non-null   float64
 6   Age                            1537 non-null   float64
 7   House_Gryffindor               1537 non-null   float64
 8   House_Hufflepuff               1537 non-null   float64
 9   House_Ravenclaw                1537 non-null   float64
 10  House_Slytherin                1537 non-null   float64
dtypes: float64(11)
memory usage: 144.1 KB


In [16]:
df_train.head(124)
# Observe que el registro de Index 121 se ha eliminada ya que 'Herbology' tenía 'NaN' 

,Defense Against the Dark Arts,Herbology,Potions,Charms,Flying,Best Hand,Age,House_Gryffindor,House_Hufflepuff,House_Ravenclaw,House_Slytherin
Index,,,,,,,,,,,
0,4.878861,5.727180,3.790369,-232.79405,-26.89,0.0,1.598905,0.0,0.0,1.0,0.0
1,5.520605,-5.987446,7.248742,-252.18425,-113.45,1.0,2.058864,0.0,0.0,0.0,1.0
2,3.660761,7.725017,8.728531,-227.34265,30.42,0.0,2.004107,0.0,0.0,1.0,0.0
3,-6.977428,-6.497214,0.821911,-256.84675,200.64,0.0,1.210130,1.0,0.0,0.0,0.0
4,-4.367752,-7.820623,4.311066,-256.38730,157.98,0.0,3.104723,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
119,-4.023001,3.530123,4.274640,-242.27829,-47.35,0.0,4.892539,0.0,1.0,0.0,0.0
120,6.051691,3.649099,8.997370,-230.00220,24.51,0.0,2.091718,0.0,0.0,1.0,0.0
122,4.362488,6.474863,4.752422,-230.41950,12.92,1.0,3.488022,0.0,0.0,1.0,0.0


# Normalización
* Realizaremos la normalización antes de la imputación con el método de los k-vecinos más próximos (KNN) ya que:
    - Es necesario normalizar las variables a una escala común antes de aplicar KNN.
    - KNN se basa en el cálculo de distancias entre observaciones. Si hay variables con escalas muy diferentes, las variables con valores más grandes dominarán el cálculo de la distancia, sesgando los resultados
* Es crucial usar EXACTAMENTE los mismos parámetros de normalización (media y desviación estándar) que se usen para los datos de entrenamiento cuando se aplique la normalización a los datos de test.
* No se deben calcular nuevos parámetros con los datos de test.
* Las variables dummy (también llamadas variables indicadoras o binarias) no deben normalizarse puesto que ya están en una escala fija (0 y 1),

### Función de Normalización

In [17]:
# Función de normalización
def normalize(column):
    mean = column.mean()
    std = column.std()
    return (column - mean) / std

### Aplicación de la Normalización a ambos DataFrames

In [18]:
# Separar las columnas a normalizar de las dummies
# 'Best Hand' no se normalizará
features_to_normalize = ['Defense Against the Dark Arts', 'Herbology', 'Potions', 
                        'Charms', 'Flying', 'Age']

# Normalizar df_train (excluyendo las variables dummy)
for column in features_to_normalize:
    mean = df_train[column].mean()
    std = df_train[column].std()
    
    # Normalizar en train
    df_train[column] = (df_train[column] - mean) / std
    
    # Usar los MISMOS parámetros para normalizar test
    df_test[column] = (df_test[column] - mean) / std

# Las variables dummy (HOUSE_FEATURES) se mantienen sin normalizar

## Guardado de los DataFrames normalizados

In [19]:
# Guardado del Datasets normalizado de entrenamiento
df_train.to_csv('../datasets/normal_train.csv')

# Guardado del Dataset de test normalizado (pre-imputación)
df_test.to_csv('../datasets/normal_test_pre_imputation.csv')

print("Dimensiones del dataset de entrenamiento:", df_train.shape)
print("Dimensiones del dataset de test:", df_test.shape)
print("\nColumnas en el dataset de entrenamiento:\n\t", df_train.columns.tolist())
print("\nColumnas en el dataset de test:\n\t", df_test.columns.tolist())

Dimensiones del dataset de entrenamiento: (1537, 11)
Dimensiones del dataset de test: (400, 7)

Columnas en el dataset de entrenamiento:
	 ['Defense Against the Dark Arts', 'Herbology', 'Potions', 'Charms', 'Flying', 'Best Hand', 'Age', 'House_Gryffindor', 'House_Hufflepuff', 'House_Ravenclaw', 'House_Slytherin']

Columnas en el dataset de test:
	 ['Defense Against the Dark Arts', 'Herbology', 'Potions', 'Charms', 'Flying', 'Best Hand', 'Age']


**Nota**
- El Dataset de test contiene 400 registros pero aún contiene algunos registros con datos faltantes.
- En el siguiente Notebook (`imputation.ipynb`) realizaremos la imputación con la técnica de los k-vecinos más próximos (KNN) [k-nearest neighbors algorithm](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)